In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [7]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [8]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [9]:
payload = payload_json

In [10]:
# engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline.build_pipeline(MSPipeline)

The following fields have not been found in any file:
['ALL_CONNECTED_PARTY_BIRTH_COUNTRIES', 'ALL_CONNECTED_GOVT_IDS', 'ALL_CONNECTED_PARTY_RESIDENCY_COUNTRIES', 'ALL_CONNECTED_PARTY_CITIZENSHIP_COUNTRIES', 'ALL_CONNECTED_TAX_IDS', 'ALL_CONNECTED_COUNTRY_OF_INCORPORATION']


## transform standardized to cleansed

In [11]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [12]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [13]:
len(new_payloads)

2

In [14]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['PL', 'Poland']


In [15]:
import pandas as pd
out_payload = pd.DataFrame([payload['watchlistParty']['matchRecords'] for payload in new_payloads])

In [16]:
out_payload[[col for col in out_payload.columns if "dobs" in col]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]"
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]"


In [17]:
from dataclasses import asdict
for i in new_payloads:
    i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}

In [18]:
import json

with open('tests/shared/parsed_payload.json', 'w') as f:
    json.dump(new_payloads, f)

In [33]:
with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [34]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [35]:
payload = payload_json

## transform standardized to cleansed

In [36]:
payload = pipeline.transform_standardized_to_cleansed(payload)

ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [37]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [38]:
len(new_payloads)

9

In [39]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
94590752
[] ['1971', '01/29/1971', '29', '01']
['AHMEDOUA'] ['FR', 'FRANCE']
PAYLOAD
94590752
[] ['27', '1986', '07', '07/27/1986']
['AHMEDOUA'] ['LIBYAN ARAB JAMAHIRIYA', 'LY']
PAYLOAD
94590752
[] ['1977', '07', '06', '07/06/1977']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08', '08/17/1938', '17', '1938']
['AHMEDOUA'] ['ID', 'INDONESIA']
PAYLOAD
94590752
[] ['15', '04/15/1958', '04', '1958']
['AHMEDOUA'] ['UNITED STATES', 'US']
PAYLOAD
94590752
[] []
['AHMEDOUA'] ['RUSSIAN FEDERATION', 'RU|PR. LENINA, D. 179']
PAYLOAD
94590752
[] ['01/01/1982', '01', '1982']
['AHMEDOUA'] ['ML|ML', 'MALI|MALI']
PAYLOAD
94590752
[] ['08', '27', '08/27/1981', '1981']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08', '08/17/1938', '17', '1938']
['AHMEDOUA'] ['ID', 'INDONESIA']


In [42]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
    except TypeError:
        pass
with open('tests/shared/big_fat_payload_parsed.json', 'w') as f:
    json.dump(new_payloads, f)

In [26]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [43]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [44]:
payload = payload_json

In [45]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [46]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [47]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
123
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['Poland', 'PL']


In [48]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [49]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
    except TypeError:
        pass
with open('tests/shared/parsed_payload_2_payload.json', 'w') as f:
    json.dump(new_payloads, f)

In [50]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [51]:
payload = payload_json

In [52]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [53]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [54]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [55]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
    except TypeError:
        pass
with open('tests/shared/empty_payload.json', 'w') as f:
    json.dump(new_payloads, f)

In [56]:
with open(f'notebooks/sample/wm_party_payload_with_partial_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [57]:
payload = payload_json

In [58]:
payload = pipeline.transform_standardized_to_cleansed(payload)

No parties
No accounts
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties


In [59]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [61]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
    except TypeError:
        pass
with open('tests/shared/wm_party_payload_with_partial_supplemental_info.json', 'w') as f:
    json.dump(new_payloads, f)


In [62]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [63]:
from etl_pipeline.config import load_agent_config

In [64]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

AttributeError: 'int' object has no attribute 'endswith'

In [65]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

NameError: name 'CONFIG_APP_DIR' is not defined

In [66]:
import pickle

In [ ]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [ ]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [ ]:
from google.protobuf.any_pb2 import Any

In [ ]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [ ]:
i.feature_inputs[0].agent_feature_input

In [ ]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

In [ ]:
message

In [ ]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


In [ ]:
NameFeatureInput.ParseFromString()

In [ ]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

In [ ]:
type(i.feature_inputs[0].agent_feature_input.type_url)

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 